In [15]:
import sys
from gamma.simulator.preprocessor import write_keywords,write_birth,write_parameters

import importlib
import gamma.simulator.preprocessor as preprocessor
importlib.reload(preprocessor)

file_name = 'cylinder.inp' #input mesh file from abaqus
toolpath_file = 'cylinder_toolpath.crs'
output_file = 'cylinder.k' #define keyword file name

substrate_height = 15
radius = 1.12
path_resolution = 0.1 # half of the element size
write_keywords(file_name,output_file,substrate_height)
write_birth(output_file,toolpath_file,path_resolution,radius,gif_end=5,nFrame=50,mode=1,camera_position=[(0, -50, 75),(0, 0, 0),(0.0, 0.0, 1.0)], make_gif=False)
write_parameters(output_file)

In [16]:
input_file = "cylinder.k"
output_file = input_file  # 원본 덮어쓰기 원하면 input_file = output_file

with open(input_file, "r") as f:
    lines = f.readlines()

modified_lines = []
inside_block = False

for i, line in enumerate(lines):
    stripped = line.strip().upper()

    # *CONTROL_TERMINATION 블록 시작 감지
    if stripped.startswith("*CONTROL_TERMINATION"):
        inside_block = True
        modified_lines.append(line)
        continue

    # $$ 주석줄은 그대로 유지
    if inside_block and line.strip().startswith("$$"):
        modified_lines.append(line)
        continue

    # 다음 줄이 실제 값이므로 해당 줄을 100으로 치환
    if inside_block:
        modified_lines.append("   100\n")  # 원하는 포맷 유지
        inside_block = False  # 블록 종료
        continue

    # 그 외 라인은 그대로
    modified_lines.append(line)

# 저장
with open(output_file, "w") as f:
    f.writelines(modified_lines)

print(f"✅ '*CONTROL_TERMINATION' 블록이 성공적으로 수정되어 '{output_file}'에 저장되었습니다.")


✅ '*CONTROL_TERMINATION' 블록이 성공적으로 수정되어 'cylinder.k'에 저장되었습니다.


In [17]:
input_file = "cylinder.k"
output_file = input_file  # 덮어쓰려면 그대로 유지

# 대체할 블록 텍스트 (줄 단위 리스트)
replacement_block = [
    "*MAT_THERMAL_ISOTROPIC_TD\n",
    "$HMNAME MATS       1MATT1_1\n",
    "         1   0.00440   1878   1928     286\n",
    "/home/ftk3187/github/GAMMA/DED_GAMMA_solver/examples/0_properties/TI64_cp.txt\n",
    "/home/ftk3187/github/GAMMA/DED_GAMMA_solver/examples/0_properties/TI64_cond.txt\n",
    "*MAT_THERMAL_ISOTROPIC_TD\n",
    "$HMNAME MATS       2MATT1_2\n",
    "         2   0.00440   1878   1928     286\n",
    "/home/ftk3187/github/GAMMA/DED_GAMMA_solver/examples/0_properties/TI64_cp.txt\n",
    "/home/ftk3187/github/GAMMA/DED_GAMMA_solver/examples/0_properties/TI64_cond.txt\n",
]

with open(input_file, "r") as f:
    lines = f.readlines()

new_lines = []
i = 0
while i < len(lines):
    line = lines[i]

    if line.strip().upper().startswith("*MAT_THERMAL_ISOTROPIC"):
        # 첫 블록만 교체 (1번만 대체되게 하기 위해)
        # 이후 블록 시작 전까지 모두 스킵
        while i < len(lines):
            i += 1
            if i < len(lines) and lines[i].strip().startswith("*") and not lines[i].strip().upper().startswith("*MAT_THERMAL_ISOTROPIC"):
                break
        new_lines.extend(replacement_block)
        # 현재 줄(* 로 시작하는 다음 블록)도 포함
        if i < len(lines):
            new_lines.append(lines[i])
            i += 1
    else:
        new_lines.append(line)
        i += 1

with open(output_file, "w") as f:
    f.writelines(new_lines)

print(f"✅ '*MAT_THERMAL_ISOTROPIC' 블록이 정확히 해당 위치에서 교체되어 저장되었습니다.")


✅ '*MAT_THERMAL_ISOTROPIC' 블록이 정확히 해당 위치에서 교체되어 저장되었습니다.


In [18]:
input_file = "cylinder.k"
output_file = input_file = "cylinder.k"

with open(input_file, "r") as f:
    lines = f.readlines()

new_lines = []
i = 0
while i < len(lines):
    line = lines[i]
    if line.strip().upper().startswith("*GAUSS_LASER"):
        new_lines.append("*GAUSS_LASER\n")
        i += 1  # 기존 설명 줄 건너뜀
        new_lines.append("500.0 1.0 0.4\n")
        i += 1
    else:
        new_lines.append(line)
        i += 1

with open(output_file, "w") as f:
    f.writelines(new_lines)

print(f"✅ '*GAUSS_LASER' 블록이 수정되어 '{output_file}'에 저장되었습니다.")


✅ '*GAUSS_LASER' 블록이 수정되어 'cylinder.k'에 저장되었습니다.


In [19]:
input_file = "cylinder.k"
output_file = input_file  # 덮어쓰기

with open(input_file, "r") as f:
    lines = f.readlines()

new_lines = []
i = 0
while i < len(lines):
    line = lines[i]
    if line.strip().upper().startswith("*CONTROL_TIMESTEP"):
        new_lines.append(line)  # 그대로 유지
        i += 1
        if i < len(lines):
            comment_line = lines[i]
            new_lines.append(comment_line)  # 주석 줄도 유지
            i += 1
        if i < len(lines):
            data_line = lines[i]
            tokens = data_line.strip().split()
            if len(tokens) > 0 and tokens[0] == "1.0E-2":
                tokens[0] = "1.0E-3"
                new_data_line = "    " + "   ".join(tokens) + "\n"
                new_lines.append(new_data_line)
            else:
                new_lines.append(data_line)
            i += 1
    else:
        new_lines.append(line)
        i += 1

with open(output_file, "w") as f:
    f.writelines(new_lines)

print(f"✅ '*CONTROL_TIMESTEP' 블록에서 DTINIT 값이 수정되어 '{output_file}'에 저장되었습니다.")


✅ '*CONTROL_TIMESTEP' 블록에서 DTINIT 값이 수정되어 'cylinder.k'에 저장되었습니다.
